<a href="https://colab.research.google.com/github/sazsa6/Learning-Deep-Learning/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Phi-3

# 1. Setup: Import Libraries

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 2. Manual Path

## 2.1. Load Model and Tokenizer

In [2]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

# Load the causal language model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False,
)

# Load the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 2.2. Tokenization Example

In [3]:
# Define the text variable
text = "This is some text to tokenize."

# Encode text into input IDs
encoding = tokenizer(text)

# Convert input IDs to readable tokens
tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])

# Print token IDs and their corresponding tokens
for token_id, token_str in zip(encoding['input_ids'], tokens):
    print(f"{token_id} --> {token_str}")

910 --> ▁This
338 --> ▁is
777 --> ▁some
1426 --> ▁text
304 --> ▁to
5993 --> ▁token
675 --> ize
29889 --> .


## 2.3. Model Inference (Logits and Prediction)

In [4]:
# Tokenize and move input to same device as model
inputs = tokenizer(text, return_tensors="pt").to(model.device)

# Get raw output logits from model
outputs = model(**inputs)

# Print max logit values (confidence per token position)
print(outputs.logits[0][0].max())  # For first token
print(outputs.logits[0][1].max())  # For second token

# Print predicted token IDs (most likely next token at each position)
print(outputs.logits[0][0].argmax())  # ID of most likely first token
print(outputs.logits[0][1].argmax())  # ID of most likely second token

# Decode a couple of token IDs manually
print(tokenizer.decode([29892, 29991]))
print(tokenizer.decode(outputs.logits[0].argmax(-1)))

# Full predicted token sequence from logits
predicted_token_id = outputs.logits.argmax(dim=-1)
print(tokenizer.decode(predicted_token_id[0]))

tensor(40.7500, device='cuda:0', dtype=torch.bfloat16, grad_fn=<MaxBackward1>)
tensor(52.2500, device='cuda:0', dtype=torch.bfloat16, grad_fn=<MaxBackward1>)
tensor(338, device='cuda:0')
tensor(263, device='cuda:0')
,!
is a code. testize.

is a code. testize.



## 2.4. Generate continuation

In [5]:
# Generate continuation
generated_ids = model.generate(
    inputs["input_ids"],
    #max_new_tokens=50,       # Number of tokens to generate
    do_sample=True,         # Use greedy decoding (set True for randomness)
    pad_token_id=tokenizer.eos_token_id  # Avoid warning if eos is not set
)

# Decode and print result
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


This is some text to tokenize. tokenize."
print("Original text: ", original_text)
print("\nTokenized


# 3. Pipeline Path

## 3.1. Pipeline Preparation

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [7]:
# Create a text-generation pipeline using the same model and tokenizer
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,   # Limit length of generated text
    do_sample=False       # Deterministic (no sampling)
)

Device set to use cuda


## 3.2. Text Generation

In [8]:
# Prompt for the model
messages = [
    {"role": "user", "content": "Hello World"}
]

# Generate text using the prompt
output = generator(messages)

# Print generated output
print(output[0]["generated_text"])

 This is a simple greeting in English, equivalent to the given Chinese greeting "你好，世界" (Hello, World). It's a common phrase used to greet someone or to acknowledge the presence of the world in a general sense.


In [9]:
# The prompt (user input / query)
messages = [
    {"role": "user", "content": "Create a funny joke about chickens."}
]

# Generate output
output = generator(messages)
print(output[0]["generated_text"])

 Why did the chicken join the band? Because it had the drumsticks!


In [10]:
# prompt: get image for this chatbot

from IPython.display import Image, display

# Option 1: Display an image from Google Drive
# Make sure you have mounted your Google Drive
# from google.colab import drive
# drive.mount('/content/drive')
# display(Image('/content/drive/My Drive/chatbot_image.png'))

# Option 2: Display an image from a URL
# Replace with the actual URL of the image
image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/04/ChatGPT_logo.svg/200px-ChatGPT_logo.svg.png'
display(Image(url=image_url))


## 3.3. Small Chatbot! Enjoy!

In [ ]:
# Store chat history
messages = []

print("Chatbot is ready! Type 'exit' to quit.\n")

while True:
    # Get user input
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print('Conversation has ended!')
        break

    # Add user message to history
    messages.append({"role": "user", "content": user_input})

    # Generate model response
    response = generator(messages)

    # Get the assistant reply
    reply = response[0]["generated_text"]
    print("Bot:", reply)

    # Add assistant reply to history
    messages.append({"role": "assistant", "content": reply})

Chatbot is ready! Type 'exit' to quit.

You: get image for chatgpt
Bot:  I'm sorry, but I can't assist with that request.
You: How can i get images?
Bot:  There are several ways to obtain images for use in a chatbot or any other application. Here are some options:

1. Public domain images: Websites like Unsplash, Pixabay, and Pexels offer high-quality images that are free to use for personal and commercial purposes.

2. Stock photo websites: Websites like Shutterstock, Adobe Stock, and Getty Images offer a wide range of images for purchase.

3. Creative Commons images: Websites like Wikimedia Commons and Flickr have a large collection of images that are available under Creative Commons licenses. You'll need to check the specific license for each image to ensure you're using it in compliance with the terms.

4. Create your own images: If you have the skills and resources, you can create your own images using graphic design software like Adobe Photoshop or Illustrator.

5. Use images fro